In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import cv2

# Training and pre-processing

loading the csv

In [2]:
data = pd.read_csv('fer2013.csv')
images = data['pixels']
labels = data['emotion']

Preprocssing



In [3]:
def preprocess_images(images):
    images = np.array([np.fromstring(image, dtype=int, sep=' ').reshape(48, 48) for image in images])
    images = np.expand_dims(images, axis=-1)
    images = images / 255.0
    return images

images = preprocess_images(images)


Split the Dataset

In [4]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)


Define the CNN Model

In [5]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')
])


Compile and Train the Model

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=24)

Epoch 1/24
898/898 [==============================] - 32s 34ms/step - loss: 1.6518 - accuracy: 0.3444 - val_loss: 1.5233 - val_accuracy: 0.4205
Epoch 2/24
898/898 [==============================] - 551s 614ms/step - loss: 1.3747 - accuracy: 0.4721 - val_loss: 1.3353 - val_accuracy: 0.4855
Epoch 3/24
898/898 [==============================] - 28s 31ms/step - loss: 1.2473 - accuracy: 0.5269 - val_loss: 1.2583 - val_accuracy: 0.5159
Epoch 4/24
898/898 [==============================] - 27s 30ms/step - loss: 1.1631 - accuracy: 0.5608 - val_loss: 1.2102 - val_accuracy: 0.5339
Epoch 5/24
898/898 [==============================] - 27s 30ms/step - loss: 1.0954 - accuracy: 0.5878 - val_loss: 1.1869 - val_accuracy: 0.5476
Epoch 6/24
898/898 [==============================] - 28s 31ms/step - loss: 1.0284 - accuracy: 0.6154 - val_loss: 1.2058 - val_accuracy: 0.5522
Epoch 7/24
898/898 [==============================] - 28s 31ms/step - loss: 0.9621 - accuracy: 0.6404 - val_loss: 1.2125 - val_accurac

Capture photo

In [7]:
# Save the Model
model.save('my_model.h5')

load the Haar cascade

In [8]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


Define the emotions

In [9]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Predit funcition

In [10]:
def predict_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = np.expand_dims(face, axis=-1)
        face = np.expand_dims(face, axis=0)
        prediction = model.predict(face)
        emotion = emotions[np.argmax(prediction)]
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    return frame


In [11]:
cap = cv2.VideoCapture(0)

Test on live video

In [ ]:
while True:
    ret, frame = cap.read()
    if ret:
        frame = predict_emotion(frame)
        cv2.imshow('Live Facial Emotion Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 19ms/step
